In [1]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import cupy as np
from NN import NeuralNetwork
from layers import DenseLayer, FlattenLayer, CnnLayer, MaxPoolingLayer
from lossFuncs import sparse_categorical_crossentropy
from activationFuncs import relu, softmax


First load the MNIST dataset. Normalize the data and split it into train and test data.

In [2]:
X, y = load_digits(return_X_y=True)
X = X.reshape(-1, 8, 8, 1)
X = X / 255
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Construct the model, the model uses sparse categorical cross entropy as its loss and an activation softmax on the output.

In [3]:
model = NeuralNetwork((8, 8, 1), lr=0.01, loss=sparse_categorical_crossentropy)
model.add_layer(CnnLayer,
                kernel_size=(4, 4),
                filters=2,
                stride=1,
                padding=1,
                activation=relu,
                input_channels=1)
model.add_layer(MaxPoolingLayer, pool_size=2, stride=2)
model.add_layer(FlattenLayer)
model.add_layer(DenseLayer, output_size=10, activation=softmax)

Train the model. We chose an epoch, this case 35 that let's the model find a low enough loss without overfitting.

In [4]:
%%time
model.train(X_train, y_train, epochs=35, batch_size=100)

Training: 100%|██████████| 35/35 [28:38<00:00, 49.11s/it, loss=0.394]

CPU times: total: 7min 13s
Wall time: 28min 39s


Evaluate the model using accuracy, precision and recall.

In [5]:
def report_metrics(y_true, y_pred):
    """Report accuracy, precision and recall for the model."""
    predicted_classes = np.argmax(y_pred, axis=1)

    true_positives = 0
    false_positives = 0
    false_negatives = 0
    true_negatives = 0

    for true_class, pred_class in zip(y_true, predicted_classes):
        if pred_class == true_class:
            if pred_class == 1:
                true_positives += 1
            else:
                true_negatives += 1
        else:
            if pred_class == 1:
                false_positives += 1
            else:
                false_negatives += 1

    # Calculating metrics
    accuracy = (true_positives + true_negatives) / len(y_test)
    precision = true_positives / (true_positives + false_positives) 
    recall = true_positives / (true_positives + false_negatives) 
    print(f"Accuracy: {accuracy*100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")

In [6]:
%%time
preds = model(X_test)
report_metrics(y_test, preds)

Accuracy: 81.94%
Precision: 0.74
Recall: 0.22
CPU times: total: 2.36 s
Wall time: 10.8 s


The model achieves a high accuracy of 81.94% and a precision of 0.74 on the MNIST dataset, indicating it is effective at identifying true positives and generally reliable when it predicts a digit class. However, the recall of 0.22 suggests it misses a significant number of actual positives, meaning it often fails to identify all instances of the correct digit classes, potentially overlooking many true digit classifications.